<a href="https://colab.research.google.com/github/harry-stark/TensorRT-BUILD-Templates/blob/main/TensorRT_Build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is Build template for building TensorRT OSS components, in a Jupyter environment.

Check for GPU and CUDA toolkit.

In [ ]:
%%shell
nvidia-smi
nvcc --version

Build or Install Cmake>3.12

In [1]:
%%shell
sudo apt install build-essential git
git clone https://github.com/Kitware/CMake/; cd CMake
./bootstrap && make && sudo make install

Streaming output truncated to the last 5000 lines.
-- Checking whether CXX compiler struct stat has st_mtim member
-- Checking whether CXX compiler struct stat has st_mtim member - yes
-- Checking whether CXX compiler struct stat has st_mtimespec member
-- Checking whether CXX compiler struct stat has st_mtimespec member - no
-- Looking for include files sys/types.h, ifaddrs.h
-- Looking for include files sys/types.h, ifaddrs.h - found
-- Checking whether CXX compiler has rlimit64
-- Checking whether CXX compiler has rlimit64 - yes
-- Looking for C++ include execinfo.h
-- Looking for C++ include execinfo.h - found
-- Checking whether backtrace works with this C++ compiler
-- Checking whether backtrace works with this C++ compiler - yes
-- Looking for C++ include dlfcn.h
-- Looking for C++ include dlfcn.h - found
-- Checking whether dladdr works with this C++ compiler
-- Checking whether dladdr works with this C++ compiler - yes
-- Looking for C++ include cxxabi.h
-- Looking for C++ inc

Install TensorRT from nvidia(https://developer.nvidia.com/tensorrt). Nvidia doesn't provide direct download access to Tensorrt . So download it to your drive.

Mount Drive and Install TensorRT

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%shell
sudo dpkg -i "/content/drive/MyDrive/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922_1-1_amd64.deb"
sudo apt-get update
sudo apt-get install tensorrt

Selecting previously unselected package nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922_1-1_amd64.deb ...
Unpacking nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922 (1-1) ...
Setting up nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922 (1-1) ...
Get:1 file:/var/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922  InRelease
Ign:1 file:/var/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922  InRelease
Get:2 file:/var/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922  Release [569 B]
Get:2 file:/var/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922  Release [569 B]
Get:3 file:/var/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922  Release.gpg [801 B]
Get:3 file:/var/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922  Release.gpg [801 B]
Get:4 htt

Clone the TensorRT OSS directory. 

In [ ]:
%%shell
git clone -b master https://github.com/nvidia/TensorRT TensorRT

Build TensorRT OSS components. Take extra care of Path variables like TRT_LIBPATH

In [ ]:
%%shell
cd TensorRT
git submodule update --init --recursive
export TRT_LIBPATH=`pwd`/TensorRT-8.2.0.6
mkdir -p build && cd build
cmake .. -DTRT_LIB_DIR=$TRT_LIBPATH -DTRT_OUT_DIR=`pwd`/out
make -j$(nproc)

Cloning into 'TensorRT'...
remote: Enumerating objects: 7169, done.
remote: Counting objects: 100% (2735/2735), done.
remote: Compressing objects: 100% (1419/1419), done.
remote: Total 7169 (delta 1390), reused 2270 (delta 1267), pack-reused 4434
Receiving objects: 100% (7169/7169), 21.15 MiB | 26.09 MiB/s, done.
Resolving deltas: 100% (4154/4154), done.
Submodule 'parsers/onnx' (https://github.com/onnx/onnx-tensorrt.git) registered for path 'parsers/onnx'
Submodule 'third_party/cub' (https://github.com/NVlabs/cub.git) registered for path 'third_party/cub'
Submodule 'third_party/protobuf' (https://github.com/protocolbuffers/protobuf.git) registered for path 'third_party/protobuf'
Cloning into '/content/TensorRT/parsers/onnx'...
Cloning into '/content/TensorRT/third_party/cub'...
Cloning into '/content/TensorRT/third_party/protobuf'...
Submodule path 'parsers/onnx': checked out '078f20b8fbcbb802aa5ffcd2134519721a61b88c'
Submodule 'third_party/onnx' (https://github.com/onnx/onnx.git) reg

Voila..TensorRT components will be inside TensorRT/build. I for one wanted onnx2trt executable. GG!

In [6]:
!pip install transformers
from pathlib import Path
from transformers import GPTNeoForCausalLM, GPT2TokenizerFast, GPTNeoConfig
from transformers.models.gpt_neo import GPTNeoOnnxConfig
from transformers.onnx.convert import export
from transformers.convert_graph_to_onnx import convert

MODEL_PATH = 'EleutherAI/gpt-neo-1.3B'
TASK = 'causal-lm'
ONNX_MODEL_PATH = Path("onnx_dir/gpt_neo_13b.onnx")
ONNX_MODEL_PATH.parent.mkdir(exist_ok=True, parents=True)
# convert(framework="pt", model=MODEL_PATH, output=ONNX_MODEL_PATH, opset=11)




def main():
    tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_PATH)
    config = GPTNeoConfig.from_pretrained(MODEL_PATH)
    onnx_config = GPTNeoOnnxConfig.with_past(config, task=TASK)

    model = GPTNeoForCausalLM(config=config).from_pretrained(MODEL_PATH)
    onnx_inputs, onnx_outputs = export(tokenizer=tokenizer, model=model, config=onnx_config, opset=12, output=ONNX_MODEL_PATH)
    print(f'Inputs: {onnx_inputs}')
    print(f'Outputs: {onnx_outputs}')


if __name__ == '__main__':
    main()

     |████████████████████████████████| 2.9 MB 8.2 MB/s 
     |████████████████████████████████| 895 kB 35.5 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 35.8 MB/s 
     |████████████████████████████████| 596 kB 57.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/gpt_neo/modeling_gpt_neo.py:558: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert batch_size > 0, "batch_size has to be defined and > 0"


Inputs: ['input_ids', 'past_key_values', 'attention_mask']
Outputs: ['logits', 'present.0.key', 'present.0.value', 'present.1.key', 'present.1.value', 'present.2.key', 'present.2.value', 'present.3.key', 'present.3.value', 'present.4.key', 'present.4.value', 'present.5.key', 'present.5.value', 'present.6.key', 'present.6.value', 'present.7.key', 'present.7.value', 'present.8.key', 'present.8.value', 'present.9.key', 'present.9.value', 'present.10.key', 'present.10.value', 'present.11.key', 'present.11.value', 'present.12.key', 'present.12.value', 'present.13.key', 'present.13.value', 'present.14.key', 'present.14.value', 'present.15.key', 'present.15.value', 'present.16.key', 'present.16.value', 'present.17.key', 'present.17.value', 'present.18.key', 'present.18.value', 'present.19.key', 'present.19.value', 'present.20.key', 'present.20.value', 'present.21.key', 'present.21.value', 'present.22.key', 'present.22.value', 'present.23.key', 'present.23.value']


In [7]:
%%shell
cp /content/TensorRT/build/parsers/onnx/onnx2trt /content



In [8]:
%%shell
export EXT_PATH=~/external
mkdir -p $EXT_PATH && cd $EXT_PATH
git clone https://github.com/pybind/pybind11.git


Cloning into 'pybind11'...
remote: Enumerating objects: 17754, done.
remote: Counting objects: 100% (1018/1018), done.
remote: Compressing objects: 100% (523/523), done.
remote: Total 17754 (delta 600), reused 778 (delta 424), pack-reused 16736
Receiving objects: 100% (17754/17754), 7.26 MiB | 15.36 MiB/s, done.
Resolving deltas: 100% (12062/12062), done.


In [50]:
%%shell
cd $EXT_PATH
cd external
mkdir "python3.8"
wget https://www.python.org/ftp/python/3.8.12/Python-3.8.12.tgz
wget http://ftp.us.debian.org/debian/pool/main/p/python3.9/libpython3.9-dev_3.9.7-4_amd64.deb

--2021-10-16 05:56:25--  https://www.python.org/ftp/python/3.8.12/Python-3.8.12.tgz
Resolving www.python.org (www.python.org)... 151.101.52.223, 2a04:4e42:d::223
Connecting to www.python.org (www.python.org)|151.101.52.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24769402 (24M) [application/octet-stream]
Saving to: ‘Python-3.8.12.tgz’

Python-3.8.12.tgz   100%[===================>]  23.62M  --.-KB/s    in 0.1s    

2021-10-16 05:56:25 (204 MB/s) - ‘Python-3.8.12.tgz’ saved [24769402/24769402]

--2021-10-16 05:56:25--  http://ftp.us.debian.org/debian/pool/main/p/python3.9/libpython3.9-dev_3.9.7-4_amd64.deb
Resolving ftp.us.debian.org (ftp.us.debian.org)... 64.50.236.52, 208.80.154.15, 64.50.233.100, ...
Connecting to ftp.us.debian.org (ftp.us.debian.org)|64.50.236.52|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4032096 (3.8M)
Saving to: ‘libpython3.9-dev_3.9.7-4_amd64.deb.1’

libpython3.9-dev_3. 100%[===================>]   3.

In [51]:
%%shell
tar zxvf /root/external/Python-3.8.12.tgz


Python-3.8.12/
Python-3.8.12/Python/
Python-3.8.12/Python/pystrtod.c
Python-3.8.12/Python/importlib.h
Python-3.8.12/Python/Python-ast.c
Python-3.8.12/Python/hamt.c
Python-3.8.12/Python/bltinmodule.c
Python-3.8.12/Python/context.c
Python-3.8.12/Python/future.c
Python-3.8.12/Python/pystrcmp.c
Python-3.8.12/Python/symtable.c
Python-3.8.12/Python/ast_unparse.c
Python-3.8.12/Python/condvar.h
Python-3.8.12/Python/getopt.c
Python-3.8.12/Python/dynload_dl.c
Python-3.8.12/Python/graminit.c
Python-3.8.12/Python/importdl.h
Python-3.8.12/Python/importlib_zipimport.h
Python-3.8.12/Python/codecs.c
Python-3.8.12/Python/pyhash.c
Python-3.8.12/Python/pymath.c
Python-3.8.12/Python/preconfig.c
Python-3.8.12/Python/dtoa.c
Python-3.8.12/Python/pystate.c
Python-3.8.12/Python/dynload_aix.c
Python-3.8.12/Python/importdl.c
Python-3.8.12/Python/strdup.c
Python-3.8.12/Python/asdl.c
Python-3.8.12/Python/compile.c
Python-3.8.12/Python/dynload_stub.c
Python-3.8.12/Python/dup2.c
Python-3.8.12/Python/opcode_targets.h

In [52]:
%%shell
rm -r /root/external/python3.9

In [53]:
%%shell
cp -r /content/Python-3.8.12/Include/ /root/external/python3.8/

In [54]:
%%shell
ar x /root/external/libpython3.9-dev_3.9.7-4_amd64.deb

In [55]:
%%shell
tar -xvf data.tar.xz

./
./usr/
./usr/bin/
./usr/bin/x86_64-linux-gnu-python3.9-config
./usr/include/
./usr/include/python3.9/
./usr/include/python3.9/Python-ast.h
./usr/include/python3.9/Python.h
./usr/include/python3.9/abstract.h
./usr/include/python3.9/asdl.h
./usr/include/python3.9/ast.h
./usr/include/python3.9/bitset.h
./usr/include/python3.9/bltinmodule.h
./usr/include/python3.9/boolobject.h
./usr/include/python3.9/bytearrayobject.h
./usr/include/python3.9/bytesobject.h
./usr/include/python3.9/cellobject.h
./usr/include/python3.9/ceval.h
./usr/include/python3.9/classobject.h
./usr/include/python3.9/code.h
./usr/include/python3.9/codecs.h
./usr/include/python3.9/compile.h
./usr/include/python3.9/complexobject.h
./usr/include/python3.9/context.h
./usr/include/python3.9/cpython/
./usr/include/python3.9/cpython/abstract.h
./usr/include/python3.9/cpython/bytearrayobject.h
./usr/include/python3.9/cpython/bytesobject.h
./usr/include/python3.9/cpython/ceval.h
./usr/include/python3.9/cpython/code.h
./usr/inclu

In [56]:
cp -r /usr/include/x86_64-linux-gnu/ /root/external/python3.8/include/

In [57]:
%%shell
export TRT_OSSPATH=/content/TensorRT

In [58]:
%%shell
cd TensorRT/python
PYTHON_MAJOR_VERSION=3 PYTHON_MINOR_VERSION=8 TARGET_ARCHITECTURE=x86_64 ./build.sh


/content/TensorRT/python/build /content/TensorRT/python
-- Configuring done
-- Generating done
-- Build files have been written to: /content/TensorRT/python/build
[  8%] Building CXX object CMakeFiles/tensorrt.dir/src/infer/pyAlgorithmSelector.cpp.o
[ 16%] Building CXX object CMakeFiles/tensorrt.dir/src/infer/pyPlugin.cpp.o
[ 25%] Building CXX object CMakeFiles/tensorrt.dir/src/infer/pyGraph.cpp.o
[ 33%] Building CXX object CMakeFiles/tensorrt.dir/src/parsers/pyCaffe.cpp.o
[ 41%] Building CXX object CMakeFiles/tensorrt.dir/src/infer/pyFoundationalTypes.cpp.o
[ 50%] Building CXX object CMakeFiles/tensorrt.dir/src/pyTensorRT.cpp.o
[ 58%] Building CXX object CMakeFiles/tensorrt.dir/src/infer/pyCore.cpp.o
[ 66%] Building CXX object CMakeFiles/tensorrt.dir/src/infer/pyInt8.cpp.o
[ 75%] Building CXX object CMakeFiles/tensorrt.dir/src/utils.cpp.o
[ 83%] Building CXX object CMakeFiles/tensorrt.dir/src/parsers/pyOnnx.cpp.o
[ 91%] Building CXX object CMakeFiles/tensorrt.dir/src/parsers/pyUff.cpp

In [65]:
%%shell
python -m pip install --upgrade pip

     |████████████████████████████████| 1.7 MB 7.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [77]:
!update-alternatives --install /usr/bin/python python /usr/bin/python3.8 1

In [76]:
!update-alternatives --list python
!sudo update-alternatives --config python
!sudo update-alternatives --set python /usr/bin/python3.8
!python3 --version

/usr/bin/python3.8
There is 1 choice for the alternative python (providing /usr/bin/python).

  Selection    Path                Priority   Status
------------------------------------------------------------
  0            /usr/bin/python3.8   1         auto mode
* 1            /usr/bin/python3.8   1         manual mode

Press <enter> to keep the current choice[*], or type selection number: 0
Python 3.7.12


In [78]:
%%shell 
python

Python 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 

KeyboardInterrupt
>>> 

CalledProcessError: ignored

In [71]:
%%shell
pip install /content/TensorRT/python/build/dist/tensorrt-8.2.0.6-cp38-none-linux_x86_64.whl

ERROR: tensorrt-8.2.0.6-cp38-none-linux_x86_64.whl is not a supported wheel on this platform.


CalledProcessError: ignored